In [1]:
import pandas as pd

In [2]:
import catboost as cb

In [3]:
df_test = pd.read_csv('correct_df.csv')

In [4]:
df_train = pd.read_csv('name_gender_features.csv')

In [29]:
df_train

,0,1,2,3,4,5,6,7,target
0,Ababij,Andrej,ij,ej,0,6,6,0,1
1,Ababilov,Dmitrij,ov,ij,1,8,7,0,1
2,Ababilov,Vladimir,ov,ir,1,8,8,0,1
3,Ababilov,Sergej,ov,ej,1,8,6,0,1
4,Ababilov,Jurij,ov,ij,1,8,5,0,1
...,...,...,...,...,...,...,...,...,...
4218084,Ksenda,Grigorij,da,ij,0,6,8,0,1
4218085,Mescherjakova,Alena,va,na,0,13,5,1,0
4218086,Shestoperov,Ivan,ov,an,1,11,4,0,1
4218087,Semenova,Efrosin'ja,va,ja,0,8,10,1,0


In [7]:
def preprocess(surname, name):
    surname_n = surname[-2:]
    name_n = name[-2:]
    
    num_let_sur = len(surname)
    num_let_nam = len(name)
    
    has_m_surend = int(surname[-2:].upper() in set(['OV', 'IN', 'AN']))
    last_let_name = int(name[-1].upper() == 'A')

    return pd.Series([
        surname,
        name,
        surname_n,
        name_n, 
        has_m_surend,
        num_let_sur,
        num_let_nam,
        last_let_name
    ])

In [30]:
df_test

,EMAIL,NAME,LAST_NAME,TEL,CITY,PASSWORD
0,I.Abramov@companyname.com,Ivan,Abramov,7776514,Moscow,392Q16t0
1,A.Nikolaev@companyname.com,Alexei,Nikolaev,1348520,Moscow,QLHczW_P
2,D.Vasilyev@companyname.com,Dmitri,Vasilyev,0,Kazan,epvEjrby
3,A.Grigoryan@companyname.com,Anastasia,Grigoryan,1928421,Yekaterinburg,No52viwx
4,A.Lisitsyn@companyname.com,Alexei,Lisitsyn,1239532,Tver,yioB4-Sc
5,D.Abramova@companyname.com,Daria,Abramova,7163908,Moscow,39FfKC6b
6,N.Kostina@companyname.com,Natalia,Kostina,9031433,Moscow,0ZTQ8;Kh
7,N.Ermolin@companyname.com,Nikolai,Ermolin,8539233,Saint petersburg,YfW2GrpL
8,V.Solovovo@companyname.com,Vladimir,Solovovo,4758395,Saint petersburg,eY5_XVPc
9,V.Ivanov@companyname.com,Vladimir,Ivanov,4827594,Novosibirsk,wMLF0H1A


In [9]:
data_test = df_test[['LAST_NAME', 'NAME']]

In [12]:
data_test = data_test.apply(lambda x: preprocess(x['LAST_NAME'], x['NAME']), axis=1)

In [13]:
data_test

,0,1,2,3,4,5,6,7
0,Abramov,Ivan,ov,an,1,7,4,0
1,Nikolaev,Alexei,ev,ei,0,8,6,0
2,Vasilyev,Dmitri,ev,ri,0,8,6,0
3,Grigoryan,Anastasia,an,ia,1,9,9,1
4,Lisitsyn,Alexei,yn,ei,0,8,6,0
5,Abramova,Daria,va,ia,0,8,5,1
6,Kostina,Natalia,na,ia,0,7,7,1
7,Ermolin,Nikolai,in,ai,1,7,7,0
8,Solovovo,Vladimir,vo,ir,0,8,8,0
9,Ivanov,Vladimir,ov,ir,1,6,8,0


In [6]:
from sklearn.model_selection import train_test_split

In [14]:
X = df_train.drop(['target'], axis=1)
y = df_train['target']

In [16]:
X_train_w, X_test_w, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [46]:
model = cb.CatBoostClassifier(depth=8, custom_loss=['AUC', 'Accuracy'])

In [47]:
cat_features = [0, 1, 2, 3]

In [48]:
model.fit(X_train_w, y_train, cat_features, eval_set=(X_test_w, y_test), use_best_model=True, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.235709
0:	learn: 0.2482512	test: 0.2481466	best: 0.2481466 (0)	total: 2s	remaining: 33m 17s
1:	learn: 0.0952745	test: 0.0948943	best: 0.0948943 (1)	total: 3.91s	remaining: 32m 32s
2:	learn: 0.0486257	test: 0.0482342	best: 0.0482342 (2)	total: 4.94s	remaining: 27m 20s
3:	learn: 0.0321198	test: 0.0317451	best: 0.0317451 (3)	total: 6.31s	remaining: 26m 10s
4:	learn: 0.0256284	test: 0.0252632	best: 0.0252632 (4)	total: 7.69s	remaining: 25m 30s
5:	learn: 0.0224923	test: 0.0221306	best: 0.0221306 (5)	total: 9.7s	remaining: 26m 47s
6:	learn: 0.0210706	test: 0.0207210	best: 0.0207210 (6)	total: 11s	remaining: 26m
7:	learn: 0.0202326	test: 0.0199019	best: 0.0199019 (7)	total: 12s	remaining: 24m 53s
8:	learn: 0.0197623	test: 0.0194439	best: 0.0194439 (8)	total: 13.5s	remaining: 24m 48s
9:	learn: 0.0195070	test: 0.0191892	best: 0.0191892 (9)	total: 15.1s	remaining: 24m 57s
10:	learn: 0.0193815	test: 0.0190582	best: 0.0190582 (10)	total: 16.2s	remaining: 24m 15s
11:	learn: 0

91:	learn: 0.0185777	test: 0.0184136	best: 0.0184136 (91)	total: 2m 56s	remaining: 28m 59s
92:	learn: 0.0185745	test: 0.0184139	best: 0.0184136 (91)	total: 2m 58s	remaining: 29m 3s
93:	learn: 0.0185745	test: 0.0184139	best: 0.0184136 (91)	total: 3m	remaining: 29m
94:	learn: 0.0185745	test: 0.0184139	best: 0.0184136 (91)	total: 3m 1s	remaining: 28m 49s
95:	learn: 0.0185745	test: 0.0184139	best: 0.0184136 (91)	total: 3m 2s	remaining: 28m 39s
96:	learn: 0.0185745	test: 0.0184139	best: 0.0184136 (91)	total: 3m 3s	remaining: 28m 30s
97:	learn: 0.0185745	test: 0.0184139	best: 0.0184136 (91)	total: 3m 4s	remaining: 28m 20s
98:	learn: 0.0185745	test: 0.0184139	best: 0.0184136 (91)	total: 3m 5s	remaining: 28m 10s
99:	learn: 0.0185745	test: 0.0184139	best: 0.0184136 (91)	total: 3m 6s	remaining: 28m 1s
100:	learn: 0.0185745	test: 0.0184139	best: 0.0184136 (91)	total: 3m 7s	remaining: 27m 52s
101:	learn: 0.0185745	test: 0.0184139	best: 0.0184136 (91)	total: 3m 9s	remaining: 27m 46s
102:	learn: 0.0

181:	learn: 0.0185670	test: 0.0184117	best: 0.0184117 (181)	total: 4m 47s	remaining: 21m 32s
182:	learn: 0.0185670	test: 0.0184117	best: 0.0184117 (182)	total: 4m 49s	remaining: 21m 30s
183:	learn: 0.0185670	test: 0.0184116	best: 0.0184116 (183)	total: 4m 50s	remaining: 21m 26s
184:	learn: 0.0185670	test: 0.0184116	best: 0.0184116 (183)	total: 4m 51s	remaining: 21m 22s
185:	learn: 0.0185670	test: 0.0184116	best: 0.0184116 (183)	total: 4m 52s	remaining: 21m 18s
186:	learn: 0.0185670	test: 0.0184116	best: 0.0184116 (183)	total: 4m 53s	remaining: 21m 16s
187:	learn: 0.0185670	test: 0.0184116	best: 0.0184116 (183)	total: 4m 55s	remaining: 21m 17s
188:	learn: 0.0185670	test: 0.0184116	best: 0.0184116 (183)	total: 4m 56s	remaining: 21m 13s
189:	learn: 0.0185670	test: 0.0184116	best: 0.0184116 (189)	total: 4m 57s	remaining: 21m 9s
190:	learn: 0.0185670	test: 0.0184116	best: 0.0184116 (190)	total: 4m 58s	remaining: 21m 5s
191:	learn: 0.0185670	test: 0.0184116	best: 0.0184116 (191)	total: 4m 59

270:	learn: 0.0185612	test: 0.0184092	best: 0.0184092 (244)	total: 6m 25s	remaining: 17m 16s
271:	learn: 0.0185612	test: 0.0184092	best: 0.0184092 (244)	total: 6m 26s	remaining: 17m 13s
272:	learn: 0.0185612	test: 0.0184092	best: 0.0184092 (244)	total: 6m 27s	remaining: 17m 11s
273:	learn: 0.0185612	test: 0.0184092	best: 0.0184092 (244)	total: 6m 28s	remaining: 17m 8s
274:	learn: 0.0185612	test: 0.0184092	best: 0.0184092 (244)	total: 6m 29s	remaining: 17m 6s
275:	learn: 0.0185612	test: 0.0184092	best: 0.0184092 (244)	total: 6m 30s	remaining: 17m 4s
276:	learn: 0.0185612	test: 0.0184092	best: 0.0184092 (244)	total: 6m 31s	remaining: 17m 1s
277:	learn: 0.0185612	test: 0.0184092	best: 0.0184092 (244)	total: 6m 32s	remaining: 16m 59s
278:	learn: 0.0185612	test: 0.0184092	best: 0.0184092 (244)	total: 6m 33s	remaining: 16m 57s
279:	learn: 0.0185612	test: 0.0184092	best: 0.0184092 (244)	total: 6m 34s	remaining: 16m 54s
280:	learn: 0.0185612	test: 0.0184092	best: 0.0184092 (244)	total: 6m 35s	

...

625:	learn: 0.0185423	test: 0.0184038	best: 0.0184029 (459)	total: 13m 17s	remaining: 7m 56s
626:	learn: 0.0185423	test: 0.0184038	best: 0.0184029 (459)	total: 13m 17s	remaining: 7m 54s
627:	learn: 0.0185423	test: 0.0184038	best: 0.0184029 (459)	total: 13m 18s	remaining: 7m 53s
628:	learn: 0.0185423	test: 0.0184038	best: 0.0184029 (459)	total: 13m 20s	remaining: 7m 51s
629:	learn: 0.0185423	test: 0.0184038	best: 0.0184029 (459)	total: 13m 21s	remaining: 7m 50s
630:	learn: 0.0185423	test: 0.0184038	best: 0.0184029 (459)	total: 13m 22s	remaining: 7m 49s
631:	learn: 0.0185423	test: 0.0184038	best: 0.0184029 (459)	total: 13m 23s	remaining: 7m 47s
632:	learn: 0.0185423	test: 0.0184038	best: 0.0184029 (459)	total: 13m 24s	remaining: 7m 46s
633:	learn: 0.0185423	test: 0.0184038	best: 0.0184029 (459)	total: 13m 25s	remaining: 7m 45s
634:	learn: 0.0185423	test: 0.0184038	best: 0.0184029 (459)	total: 13m 26s	remaining: 7m 43s
635:	learn: 0.0185423	test: 0.0184038	best: 0.0184029 (459)	total: 13m

714:	learn: 0.0185423	test: 0.0184038	best: 0.0184029 (459)	total: 14m 53s	remaining: 5m 56s
715:	learn: 0.0185423	test: 0.0184038	best: 0.0184029 (459)	total: 14m 54s	remaining: 5m 54s
716:	learn: 0.0185423	test: 0.0184038	best: 0.0184029 (459)	total: 14m 55s	remaining: 5m 53s
717:	learn: 0.0185423	test: 0.0184038	best: 0.0184029 (459)	total: 14m 56s	remaining: 5m 52s
718:	learn: 0.0185423	test: 0.0184038	best: 0.0184029 (459)	total: 14m 57s	remaining: 5m 50s
719:	learn: 0.0185422	test: 0.0184037	best: 0.0184029 (459)	total: 14m 58s	remaining: 5m 49s
720:	learn: 0.0185422	test: 0.0184037	best: 0.0184029 (459)	total: 14m 59s	remaining: 5m 48s
721:	learn: 0.0185422	test: 0.0184037	best: 0.0184029 (459)	total: 15m 1s	remaining: 5m 47s
722:	learn: 0.0185422	test: 0.0184037	best: 0.0184029 (459)	total: 15m 2s	remaining: 5m 45s
723:	learn: 0.0185422	test: 0.0184037	best: 0.0184029 (459)	total: 15m 4s	remaining: 5m 44s
724:	learn: 0.0185422	test: 0.0184037	best: 0.0184029 (459)	total: 15m 5s

803:	learn: 0.0185400	test: 0.0184025	best: 0.0184025 (764)	total: 16m 36s	remaining: 4m 2s
804:	learn: 0.0185400	test: 0.0184025	best: 0.0184025 (764)	total: 16m 37s	remaining: 4m 1s
805:	learn: 0.0185400	test: 0.0184025	best: 0.0184025 (764)	total: 16m 38s	remaining: 4m
806:	learn: 0.0185400	test: 0.0184025	best: 0.0184025 (764)	total: 16m 39s	remaining: 3m 59s
807:	learn: 0.0185400	test: 0.0184025	best: 0.0184025 (764)	total: 16m 40s	remaining: 3m 57s
808:	learn: 0.0185400	test: 0.0184025	best: 0.0184025 (764)	total: 16m 41s	remaining: 3m 56s
809:	learn: 0.0185400	test: 0.0184025	best: 0.0184025 (764)	total: 16m 42s	remaining: 3m 55s
810:	learn: 0.0185400	test: 0.0184025	best: 0.0184025 (764)	total: 16m 43s	remaining: 3m 53s
811:	learn: 0.0185400	test: 0.0184025	best: 0.0184025 (764)	total: 16m 45s	remaining: 3m 52s
812:	learn: 0.0185400	test: 0.0184025	best: 0.0184025 (764)	total: 16m 46s	remaining: 3m 51s
813:	learn: 0.0185400	test: 0.0184025	best: 0.0184025 (764)	total: 16m 47s	r

892:	learn: 0.0185341	test: 0.0184031	best: 0.0184025 (764)	total: 18m 18s	remaining: 2m 11s
893:	learn: 0.0185341	test: 0.0184031	best: 0.0184025 (764)	total: 18m 19s	remaining: 2m 10s
894:	learn: 0.0185341	test: 0.0184031	best: 0.0184025 (764)	total: 18m 20s	remaining: 2m 9s
895:	learn: 0.0185341	test: 0.0184031	best: 0.0184025 (764)	total: 18m 21s	remaining: 2m 7s
896:	learn: 0.0185341	test: 0.0184031	best: 0.0184025 (764)	total: 18m 22s	remaining: 2m 6s
897:	learn: 0.0185341	test: 0.0184031	best: 0.0184025 (764)	total: 18m 24s	remaining: 2m 5s
898:	learn: 0.0185341	test: 0.0184031	best: 0.0184025 (764)	total: 18m 25s	remaining: 2m 4s
899:	learn: 0.0185340	test: 0.0184031	best: 0.0184025 (764)	total: 18m 26s	remaining: 2m 2s
900:	learn: 0.0185340	test: 0.0184030	best: 0.0184025 (764)	total: 18m 27s	remaining: 2m 1s
901:	learn: 0.0185340	test: 0.0184030	best: 0.0184025 (764)	total: 18m 28s	remaining: 2m
902:	learn: 0.0185340	test: 0.0184030	best: 0.0184025 (764)	total: 18m 29s	remain

981:	learn: 0.0185288	test: 0.0184001	best: 0.0184001 (980)	total: 19m 58s	remaining: 22s
982:	learn: 0.0185288	test: 0.0184001	best: 0.0184001 (982)	total: 19m 59s	remaining: 20.7s
983:	learn: 0.0185288	test: 0.0184001	best: 0.0184001 (983)	total: 20m	remaining: 19.5s
984:	learn: 0.0185288	test: 0.0184001	best: 0.0184001 (984)	total: 20m 1s	remaining: 18.3s
985:	learn: 0.0185288	test: 0.0184001	best: 0.0184001 (984)	total: 20m 2s	remaining: 17.1s
986:	learn: 0.0185287	test: 0.0184001	best: 0.0184001 (986)	total: 20m 3s	remaining: 15.9s
987:	learn: 0.0185287	test: 0.0184001	best: 0.0184001 (987)	total: 20m 4s	remaining: 14.6s
988:	learn: 0.0185287	test: 0.0184001	best: 0.0184001 (988)	total: 20m 5s	remaining: 13.4s
989:	learn: 0.0185287	test: 0.0184001	best: 0.0184001 (989)	total: 20m 7s	remaining: 12.2s
990:	learn: 0.0185287	test: 0.0184001	best: 0.0184001 (990)	total: 20m 8s	remaining: 11s
991:	learn: 0.0185287	test: 0.0184001	best: 0.0184001 (991)	total: 20m 9s	remaining: 9.75s
992:

In [49]:
df_test['gender'] = model.predict(data_test)

In [50]:
model.predict_proba(data_test)[17]

array([0.54920935, 0.45079065])

In [54]:
df_test

,EMAIL,NAME,LAST_NAME,TEL,CITY,PASSWORD,gender
0,I.Abramov@companyname.com,Ivan,Abramov,7776514,Moscow,392Q16t0,1
1,A.Nikolaev@companyname.com,Alexei,Nikolaev,1348520,Moscow,QLHczW_P,1
2,D.Vasilyev@companyname.com,Dmitri,Vasilyev,0,Kazan,epvEjrby,1
3,A.Grigoryan@companyname.com,Anastasia,Grigoryan,1928421,Yekaterinburg,No52viwx,0
4,A.Lisitsyn@companyname.com,Alexei,Lisitsyn,1239532,Tver,yioB4-Sc,1
5,D.Abramova@companyname.com,Daria,Abramova,7163908,Moscow,39FfKC6b,0
6,N.Kostina@companyname.com,Natalia,Kostina,9031433,Moscow,0ZTQ8;Kh,0
7,N.Ermolin@companyname.com,Nikolai,Ermolin,8539233,Saint petersburg,YfW2GrpL,1
8,V.Solovovo@companyname.com,Vladimir,Solovovo,4758395,Saint petersburg,eY5_XVPc,1
9,V.Ivanov@companyname.com,Vladimir,Ivanov,4827594,Novosibirsk,wMLF0H1A,1


In [56]:
model.save_model('gender_predictor.cbm')